In [1]:
import os
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [3]:
data_path = r'D:\Projects\superparcels\data\abreunig_temp_sp_sample_08013.csv'

In [6]:
data_dir = r'D:\Projects\superparcels\data'

In [ ]:
df = pd.read_csv(data_path)
df.head()

In [7]:
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'].apply(wkt.loads), crs=4326)
gdf.to_file(os.path.join(data_dir, 'sp_sample_08013.shp'))

Clean parcel urban and remove duplicate geometries for clustering POC

In [34]:
input_dir = r'D:\Projects\superparcels\data\urban'

In [56]:
df = gpd.read_file(os.path.join(input_dir, 'sp_sample_08013_urban.shp'))

In [57]:
df.head(3)

,state_code,cnty_code,OWNER,std_addr,geometry
0,8,13,ALEAKOOTZ,100 RIDGE DR,"POLYGON ((-105.30127 40.06144, -105.30126 40.0..."
1,8,13,MARIJETERELLEN,501 EVERGREEN AVE,"POLYGON ((-105.28974 40.03127, -105.28974 40.0..."
2,8,13,DENNIS VMARTIN,2940 18TH ST,"POLYGON ((-105.27360 40.02916, -105.27359 40.0..."


In [58]:
df.dissolve(by='OWNER').to_file(os.path.join(input_dir, 'sp_sample_08013_urban_dissolved.shp'))

In [55]:
df = df.dissolve(by='OWNER').explode(ignore_index=False).reset_index()

C:\Users\abreunig\AppData\Local\Temp\2\ipykernel_14072\1990443479.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  df = df.dissolve(by='OWNER').explode(ignore_index=False).reset_index()


In [48]:
df.head(3)

,OWNER,level_1,state_code,cnty_code,std_addr,duplicate_owner,duplicate_address,duplicate_geometry,classification,classification_code,geometry
0,1827GROVE,0,8,13,1827 GROVE ST,False,False,False,Unique Owner & Address & Geometry,3,"POLYGON ((-105.27067 40.01550, -105.27071 40.0..."
1,2004 DAVID BROWN BOULDER RESIDENTIAL (TE),0,8,13,3175 10TH ST,False,False,False,Unique Owner & Address & Geometry,3,"POLYGON ((-105.28454 40.03244, -105.28454 40.0..."
2,2004 JAMES THOMASCHRISTOL,0,8,13,1200 YARMOUTH AVE UNIT 231,True,True,True,"Duplicate Owner, Address & Geometry",2,"POLYGON ((-105.28198 40.05631, -105.28198 40.0..."


In [49]:
#import pandas as pd

# Sample DataFrame
#data = {'owner': ['A', 'B', 'A', 'C', 'D', 'E', 'E', 'F'],
#        'address': ['X', 'Y', 'X', 'Z', 'W', 'Y', 'W', 'Z'],
#        'geometry': ['geom1', 'geom2', 'geom1', 'geom3', 'geom4', 'geom2', 'geom4', 'geom3']}
#df = pd.DataFrame(data)

# Identify duplicate owners, addresses, and geometries
df['duplicate_owner'] = df.duplicated(subset=['OWNER'], keep=False)
df['duplicate_address'] = df.duplicated(subset=['std_addr'], keep=False)
df['duplicate_geometry'] = df.duplicated(subset=['geometry'], keep=False)

# Create a classification column based on duplication status (with geometry)
df['classification'] = df.apply(
    lambda row: (
        'Duplicate Owner, Address & Geometry' if row['duplicate_owner'] and row['duplicate_address'] and row['duplicate_geometry'] else
        'Duplicate Owner & Address, Unique Geometry' if row['duplicate_owner'] and row['duplicate_address'] and not row['duplicate_geometry'] else
        'Duplicate Owner & Unique Address, Geometry' if row['duplicate_owner'] and not row['duplicate_address'] and not row['duplicate_geometry'] else
        'Unique Owner & Duplicate Address & Geometry' if not row['duplicate_owner'] and row['duplicate_address'] and row['duplicate_geometry'] else
        'Unique Owner & Address, Duplicate Geometry' if not row['duplicate_owner'] and not row['duplicate_address'] and row['duplicate_geometry'] else
        'Unique Owner & Address & Geometry'
    ), axis=1
)

# Count the occurrences of each classification
matrix = df['classification'].value_counts()

print(matrix)


classification
Unique Owner & Address & Geometry              14000
Unique Owner & Address, Duplicate Geometry      5818
Duplicate Owner & Address, Unique Geometry      2405
Duplicate Owner, Address & Geometry             1636
Unique Owner & Duplicate Address & Geometry       16
Name: count, dtype: int64


In [50]:
# create clasification codes
df['classification_code'] = df['classification'].astype('category').cat.codes
df['classification_code'].value_counts()

classification_code
2    14000
3     5818
0     2405
1     1636
4       16
Name: count, dtype: int64

In [51]:
class_code = 0
code = df[df['classification_code'] == class_code]
code.to_file(os.path.join(input_dir, f'sp_sample_08013_code{class_code}_dissolved.shp'))

C:\Users\abreunig\AppData\Local\Temp\2\ipykernel_14072\872312086.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  code.to_file(os.path.join(input_dir, f'sp_sample_08013_code{class_code}_dissolved.shp'))


In [52]:
cluster_canidate_codes = [0 , 1] # classes after dissolving
cluster = df[df['classification_code'].isin(cluster_canidate_codes)]
cluster.to_file(os.path.join(input_dir, f'sp_sample_08013_cluster_canidates.shp'))

C:\Users\abreunig\AppData\Local\Temp\2\ipykernel_14072\3598197704.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  cluster.to_file(os.path.join(input_dir, f'sp_sample_08013_cluster_canidates.shp'))
